In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

    0 = spontanic
    1 = sync
    2 = alone

In [206]:
def encode(path, num):
    df = pd.read_csv(path).iloc[8:,:]
    for i in range(len(df)):
        df['label']=num
        
      
    for col in df.columns:
        col1 = col.replace(" ", "_")
        df = df.rename({col: col1}, axis=1)
        
    df = df.rename({'_#_hands': 'hands_num'}, axis=1)  
    
    
    if(num != 2):
        df = df[df.hands_num == 2]
    else:
        df = df[df.hands_num == 1]
    left, right =split(df)
    rename(left, 'L')
    rename(right, 'R')
    df = pd.concat([left, right], axis=1)
    return df

In [207]:


def split(df) :
    half = int(len(df)/2)
    hd = df.head(half)
    tl = df.tail(len(df)-half)
    hd = hd.reset_index(drop=True)
    tl = tl.reset_index(drop=True)
    return hd, tl


In [208]:
def rename(df, char):
    for col in df.columns:
        col1 = char+col
        df = df.rename({col: col1}, axis=1)

In [197]:
left = left.reset_index(drop=True)
right = right.reset_index(drop=True)

In [198]:
result = pd.concat([left, right], axis=1)
result

,LlTime,Ll_Frame_ID,Ll_Hand_Type,Llhands_num,Ll_Position_X,Ll_Position_Y,Ll_Position_Z,Ll_Velocity_X,Ll_Velocity_Y,Ll_Velocity_Z,...,_Wrist_Pos_X,_Wrist_Pos_Y,_Wrist_Pos_Z,_Elbow_pos_X,_Elbow_Pos_Y,_Elbow_Pos_Z,_Grab_Strenth,_Grab_Angle,_Pinch_Strength,label
0,410.9792,137197,left,2,-102.09200,198.4801,36.821470,-143.442100,162.222500,138.818300,...,205.9609,175.6849,14.267460,350.4404,35.94054,155.31430,0.000000,0.000000,0.00000,0
1,367.0280,132133,left,2,-87.15145,176.7649,2.256261,-7.801042,23.199360,5.160039,...,204.5794,195.9741,10.362830,319.3532,40.66105,161.77370,0.000000,0.743817,0.00000,0
2,395.4901,135412,left,2,-84.49631,110.0742,0.734960,-2.796520,0.579719,25.819580,...,204.3465,171.5829,29.251980,336.9697,32.85902,184.93140,0.019736,1.157780,1.00000,0
3,367.0112,132131,left,2,-87.14587,176.6150,2.159291,0.736644,-5.149579,3.776033,...,200.3514,225.1429,-10.315050,312.5537,48.40883,118.97450,0.000000,1.182807,0.52701,0
4,366.9946,132129,left,2,-87.10238,176.6544,2.143091,-8.262551,10.579970,-3.158987,...,159.9271,274.3244,20.150710,315.2328,97.94558,92.10949,0.000000,0.224456,0.00000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4518,403.4247,136327,left,2,-92.03870,138.9206,22.737920,-64.401780,287.341600,75.787410,...,193.8859,161.2190,-18.100470,365.6579,67.07969,134.63960,0.000000,0.382699,0.00000,0
4519,352.9461,130510,left,2,-61.84723,188.6001,-2.761296,12.823170,34.345700,7.999161,...,192.8679,160.3457,-18.442610,362.9705,61.52768,133.20670,0.000000,0.327827,0.00000,0
4520,402.6129,136233,left,2,-100.04990,173.2896,2.224796,-12.754790,169.522500,61.590700,...,192.1626,159.6837,-18.455740,358.7907,56.64771,134.24010,0.000000,0.232746,0.00000,0
4521,352.8800,130502,left,2,-63.17524,185.0575,-1.767847,6.663725,62.045470,-56.591190,...,194.4115,156.0282,-14.795620,372.7039,87.74497,144.09610,0.933632,2.060410,0.00000,0


In [209]:
#evyatar
ev_spo = encode(r'Evyatar636771052727603804Spontan.csv',0)
ev_sync = encode(r'Evyatar636771053639929594Sync.csv',1)
ev_al = encode(r'Evyatar636771054555711409Alone.csv',2)

In [210]:
ev_sync

,Time,_Frame_ID,_Hand_Type,hands_num,_Position_X,_Position_Y,_Position_Z,_Velocity_X,_Velocity_Y,_Velocity_Z,...,_Wrist_Pos_X,_Wrist_Pos_Y,_Wrist_Pos_Z,_Elbow_pos_X,_Elbow_Pos_Y,_Elbow_Pos_Z,_Grab_Strenth,_Grab_Angle,_Pinch_Strength,label
0,427.2973,139078,right,2,68.29678,230.0778,-23.70698,3.900916,374.071900,-87.240550,...,180.0193,174.3923,63.44727,367.8370,19.281230,120.16880,0.846882,1.241262,0.884765,1
1,427.2973,139078,left,2,-83.72626,224.9673,-44.57369,-24.301780,392.917100,-43.769510,...,-113.6673,131.3813,51.22487,-332.6850,10.864050,43.53605,0.743367,2.234670,1.000000,1
2,427.3138,139080,right,2,68.64410,236.4162,-24.91455,39.698980,380.604700,-69.140550,...,180.3322,173.8456,63.05048,365.4636,14.917780,118.01040,0.872050,1.261028,0.896960,1
3,427.3138,139080,left,2,-83.96197,229.8747,-44.67616,-3.998017,245.686600,6.163967,...,-111.8278,130.5845,51.55807,-326.0891,1.673831,46.42205,0.580397,2.287038,1.000000,1
4,427.3304,139082,right,2,69.02287,242.8502,-25.98073,26.363270,392.009800,-59.929800,...,181.1560,173.7654,62.85649,363.9774,11.971130,117.18160,0.883549,1.263857,0.901192,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4517,464.7093,143388,left,2,-76.25923,184.4595,58.09048,139.877000,-4.876449,-13.017560,...,-126.9241,191.1363,58.91284,-302.6545,12.995740,26.75648,0.000000,0.000000,0.000000,1
4518,464.7258,143390,right,2,116.90140,204.9822,52.65088,5.495839,-4.981175,-5.686485,...,153.5864,150.0203,78.10875,363.4342,50.553340,176.69620,0.000000,0.144381,0.000000,1
4519,464.7258,143390,left,2,-75.22142,186.1176,58.57318,49.763750,102.934100,33.203420,...,-126.4018,188.6597,59.13661,-308.9988,15.970830,37.08796,0.000000,0.000000,0.000000,1
4520,464.7424,143392,right,2,117.25860,205.2534,52.58410,26.987550,24.853150,-2.190735,...,153.5919,150.3768,77.51683,364.0365,52.970890,176.88680,0.000000,0.148039,0.000000,1


In [133]:
#nofar
nofar_spo = encode(r'Nofar636759795182793299Spontan.csv',0)
nofar_sync = encode(r'Nofar636759797397919664Sync.csv',1)
nofar_al = encode(r'Nofar636759796290435160Alone.csv',2)

In [134]:
#oriya
oriya_spo = encode(r'Oriya636759804404113837Spontan.csv',0)
oriya_sync = encode(r'Oriya636759806131350399Sync.csv',1)
oriya_al = encode(r'Oriya636759805268396661Alone.csv',2)


In [135]:
#orya
orya_spo = encode(r'OryaB636771083605535985Spontan.csv',0)
orya_sync = encode(r'OryaB636771084494502229Sync.csv',1)
orya_al = encode(r'OryaB636771082736601528Alone.csv',2)


In [136]:
#revital
rev_spo = encode(r'Revital636770957038873193Spontan.csv',0)
rev_sync = encode(r'Revital636770957894370062Sync.csv',1)
rev_al = encode(r'Revital636770955999435879Alone.csv',2)


In [137]:
#sachar
sachar_spo = encode(r'Sachar636771045591258071Spontan.csv',0)
sachar_sync = encode(r'Sachar636771046527112972Sync.csv',1)
sachar_al = encode(r'Sachar636771047408504303Alone.csv',2)


In [138]:
#shelly
shelly_spo = encode(r'Shelly636759787508979097Spontan.csv',0)
shelly_sync = encode(r'Shelly636759789526313406Sync.csv',1)
shelly_al = encode(r'Shelly636759788383205274Alone.csv',2)


In [139]:
#yael
yael_spo = encode(r'Yael636771059544754823Spontan.csv',0)
yael_sync = encode(r'Yael636771060699529085Sync.csv',1)
yael_al = encode(r'Yael636771061659067649Alone.csv',2)


In [191]:
#yoel
yoel_spo = encode(r'Yoel636760921663075813Spontan.csv',0)
yoel_sync = encode(r'Yoel636760923490742912Sync.csv',1)
yoel_al = encode(r'Yoel636760922573364655Alone.csv',2)

In [100]:
dataFrames = [ev_spo, nofar_spo, oriya_spo, orya_spo,rev_spo, sachar_spo, shelly_spo, yael_spo, yoel_spo,
         ev_sync, nofar_sync, oriya_sync, orya_sync,rev_sync, sachar_sync, shelly_sync, yael_sync, yoel_sync,
         ev_al, nofar_al, oriya_al, orya_al,rev_al, sachar_al, shelly_al, yael_al, yoel_al ]

df = pd.concat(dataFrames)

In [101]:
df

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength,label
7,336.0481,128562,left,2,-95.37997,156.0973,-40.18840,-3.056150,203.812100,19.494730,...,-148.3562,118.3093,-10.91977,-314.2976,-49.04110,81.41824,0.000000,0.000000,0.000000,0
8,336.0651,128564,right,2,132.46290,216.8003,-25.45043,98.732320,-65.536460,-218.720100,...,179.5375,165.1393,-15.53189,271.5673,-37.88867,104.38140,0.000000,0.728858,0.000000,0
9,336.0651,128564,left,2,-97.10074,161.7751,-40.36338,-98.927470,316.645200,-7.328697,...,-149.2292,121.7602,-12.78066,-299.9106,-56.55378,85.03669,0.000000,0.000000,0.000000,0
10,336.0816,128566,right,2,134.08500,214.4446,-28.93817,138.872500,-93.848970,-209.651200,...,181.0919,163.4637,-15.32005,268.2274,-43.06868,102.24200,0.000000,0.802890,0.000000,0
11,336.0816,128566,left,2,-98.59742,166.4238,-40.22689,-40.499030,197.513500,28.777420,...,-150.8117,125.7308,-14.01807,-293.6814,-57.60085,86.21560,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,764.4578,59984,left,1,-125.81370,214.8894,20.59324,20.170870,19.997410,37.082170,...,-146.6314,148.2971,44.40766,-258.0887,-77.52971,112.97060,0.116068,1.901876,0.000000,2
2246,764.4913,59986,left,1,-125.92320,215.6769,22.09264,-6.867068,21.957800,42.485630,...,-146.1113,148.8045,45.68930,-253.6847,-80.13625,109.99670,0.134497,1.952747,0.000000,2
2247,764.5253,59988,left,1,-125.62170,215.7428,23.39750,17.454730,-3.217659,31.446690,...,-144.8174,148.4588,46.66075,-249.6531,-83.33176,105.00430,0.120680,1.984825,0.000000,2
2248,764.5587,59989,left,1,-125.13520,215.6353,23.80248,25.567410,-5.648659,21.284490,...,-143.8482,148.1024,46.72861,-248.2768,-84.26936,103.46830,0.097735,1.988376,0.000000,2


In [104]:
df.isna().sum()

Time               0
 Frame ID          0
 Hand Type         0
 # hands           0
 Position X        0
 Position Y        0
 Position Z        0
 Velocity X        0
 Velocity Y        0
 Velocity Z        0
 Pitch             0
 Roll              0
 Yaw               0
 Wrist Pos X       0
 Wrist Pos Y       0
 Wrist Pos Z       0
 Elbow pos X       0
 Elbow Pos Y       0
 Elbow Pos Z       0
 Grab Strenth      0
 Grab Angle        0
 Pinch Strength    0
label              0
dtype: int64